In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('insert_your_data_path', one_hot=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting /Users/urim/Data/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/urim/Data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/urim/Data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/urim/Data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
B = 50
T = 28
F = 28
o_size = 10 #output_size
h1 = 128

In [3]:
inputs = tf.placeholder(tf.float32, shape=[None, T, F]) #B,T,F
labels = tf.placeholder(tf.float32, shape=[None, 10])
seq_length = tf.placeholder(tf.float32, shape=[None])

W1 = tf.Variable(tf.random_normal([h1,o_size]))
b1 = tf.Variable(tf.random_normal([o_size]))

#cell = tf.nn.rnn_cell.LSTMCell(h1) #multirnncell에서 [cell]*2로 하면 계산이 되서 에러가 남
#cell_2 = tf.nn.rnn_cell.LSTMCell(h1)
#cell_ls = tf.nn.rnn_cell.MultiRNNCell([cell, cell_2])
cell_ls = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(h1) for i in range(3)])

outputs, states = tf.nn.dynamic_rnn(cell_ls, inputs, sequence_length=seq_length, dtype=tf.float32) #outpus.shape = [B,T,F]

output_last = outputs[:,-1,:] #[B,T(last),H] -> [B,H]

linear = tf.matmul(output_last, W1) + b1 #[B,H] * [H,outsize] 

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=linear, labels=labels))
train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf.nn.softmax(linear), axis=1), tf.argmax(labels, axis=1)), tf.float32))

In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(1000 + 1):
        
        batch_tr_inputs, batch_tr_labels = mnist.train.next_batch(B)
        reshape_batch_tr_inputs = batch_tr_inputs.reshape([B, T, F])
        _, tr_loss, tr_acc = sess.run([train_step, loss, acc], 
                                      feed_dict={inputs: reshape_batch_tr_inputs, labels: batch_tr_labels, seq_length: [T]*B}) #[28,28,28,...].shape = ()
        
        batch_val_inputs, batch_val_labels = mnist.validation.next_batch(B)
        reshape_batch_val_inputs = batch_val_inputs.reshape([B, T, F])
        val_loss, val_acc = sess.run([loss, acc], 
                                     feed_dict={inputs: reshape_batch_val_inputs, labels: batch_val_labels, seq_length: [T]*B})
        
        batch_te_inputs, batch_te_labels = mnist.test.next_batch(B)
        reshape_batch_te_inputs = batch_te_inputs.reshape([B, T, F])
        te_loss, te_acc = sess.run([loss, acc], 
                                     feed_dict={inputs: reshape_batch_te_inputs, labels: batch_te_labels, seq_length: [T]*B})
        
        if step % 100 == 0:
            print('step: {} \ntr_loss: {:0.4f} \ttr_acc: {:0.4f} \nval_loss: {:0.4f} \tval_acc: {:0.4f} \nte_loss: {:0.4f} \tte_acc: {:0.4f}\n'
                  .format(step, tr_loss, tr_acc, val_loss, val_acc, te_loss, te_acc))

step: 0 
tr_loss: 3.0511 	tr_acc: 0.1000 
val_loss: 2.4789 	val_acc: 0.0500 
te_loss: 2.6713 	te_acc: 0.0500

step: 100 
tr_loss: 1.3421 	tr_acc: 0.4500 
val_loss: 0.8327 	val_acc: 0.6000 
te_loss: 0.6218 	te_acc: 0.8000

step: 200 
tr_loss: 0.5163 	tr_acc: 0.8500 
val_loss: 0.3332 	val_acc: 0.9000 
te_loss: 0.4817 	te_acc: 0.8500

step: 300 
tr_loss: 0.4018 	tr_acc: 0.8000 
val_loss: 0.3154 	val_acc: 0.8500 
te_loss: 0.5195 	te_acc: 0.8500

step: 400 
tr_loss: 0.2711 	tr_acc: 0.8500 
val_loss: 0.9545 	val_acc: 0.7500 
te_loss: 0.1637 	te_acc: 0.9500

step: 500 
tr_loss: 0.0566 	tr_acc: 1.0000 
val_loss: 0.3513 	val_acc: 0.9000 
te_loss: 0.1983 	te_acc: 0.8500

step: 600 
tr_loss: 0.3324 	tr_acc: 0.9000 
val_loss: 0.1823 	val_acc: 0.9000 
te_loss: 0.1854 	te_acc: 0.9000

step: 700 
tr_loss: 0.2129 	tr_acc: 0.9500 
val_loss: 0.0642 	val_acc: 1.0000 
te_loss: 0.2124 	te_acc: 0.9500

step: 800 
tr_loss: 0.4352 	tr_acc: 0.8500 
val_loss: 0.1033 	val_acc: 0.9500 
te_loss: 0.0621 	te_acc: 1.

In [4]:
value = [1,2,3]
test_tensor = tf.constant(value, dtype=tf.float32)